In [1]:
SAVE_FIGURES = false;

In [2]:
using Revise, DiffEqSensitivity, Enzyme, ForwardDiff, Polyester
using Plots, Statistics, BioSimulator, DataFrames, DifferentialEquations, Random, LinearAlgebra
using ForwardDiff: Chunk, JacobianConfig

In [3]:
using AnalyticSensitivity
ASPkg = AnalyticSensitivity;

┌ Info: Precompiling AnalyticSensitivity [e1d15a82-05cc-4acd-a41f-95fd5ab75271]
└ @ Base loading.jl:1342


In [4]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)
Environment:
  JULIA_NUM_THREADS = 4


# Figure 1

In [ ]:
iters = 10
t = 1000.0
tspan = (0.0, t)
k1 = zeros(iters)
k2 = zeros(iters)
X = zeros(iters, iters)
Y = zeros(iters, iters)
x0 = [1.25e4, 6.25e2] # initial values
for i = 1:iters
    k1[i] = 5.5e-9 + (i - 1) * (1.0e-11)
    for j = 1:iters
        k2[j] = 2.5e-11 + (j - 1) * (1.0e-13)
        p = [k1[i], k2[j], 1.0e-6, 6.0e-2, 1.0e9]
        prob = ODEProblem(CARRGO, x0, tspan, p)
        sol = solve(prob)
        X[i,j]= sol(tspan[2])[1]
        Y[i,j]= sol(tspan[2])[2]
    end
end
CancerCellsHeatmap=Plots.heatmap(k1, k2, X,xaxis="k1",yaxis="k2",title="Number of Cancer Cells")
SAVE_FIGURES && savefig("CancerCellsHeatmap")
ImmuneCellsHeatmap=Plots.heatmap(k1, k2, Y,xaxis="k1",yaxis="k2",title="Number of Immune Cells")
SAVE_FIGURES && savefig("ImmuneCellsHeatmap")
display(CancerCellsHeatmap)
display(ImmuneCellsHeatmap)

# Figure 2

In [ ]:
p = [6.0e-9, 3.0e-11, 1.0e-6, 6.0e-2, 1.0e9]; # parameters
x0 = [1.25e4, 6.25e2]; # initial values
(d, tspan) = (1.0e-12, (0.0,1000.0)); # 1000 days
solution = analytic_method(Order1(), CARRGO, x0, p, tspan, d, alg=Vern9(), abstol=1e-12, reltol=1e-12); # find solution and partials
CARRGO1 = plot(solution[1][:, 1], label = "x1", xlabel= "days", 
ylabel = "cancer cells x1", xlims = (tspan[1],tspan[2]), dpi = 120, linewidth = 3)
SAVE_FIGURES && savefig("CARRGO1")
CARRGO2 = plot(solution[1][:, 2], label = "d1x1", xlabel= "days", 
ylabel = "p1 sensitivity", xlims = (tspan[1],tspan[2]), dpi = 120, linewidth = 3)
SAVE_FIGURES && savefig("CARRGO2")
CARRGO3 = plot(solution[1][:, 3], label = "d2x1", xlabel= "days",
ylabel = "p2 sensitivity", xlims = (tspan[1],tspan[2]), dpi = 120, linewidth = 3)
SAVE_FIGURES && savefig("CARRGO3")
CARRGO4 = plot(solution[1][:, 4], label = "d3x1", xlabel= "days",
ylabel = "p3 sensitivity", xlims = (tspan[1],tspan[2]), dpi = 120, linewidth = 3)
SAVE_FIGURES && savefig("CARRGO4")
CARRGO5 = plot(solution[1][:, 5], label = "d4x1", xlabel= "days",
ylabel = "p4 sensitivity", xlims = (tspan[1],tspan[2]), dpi = 120, linewidth = 3)
SAVE_FIGURES && savefig("CARRGO5")
CARRGO6 = plot(solution[1][:, 6], label = "d5x1", xlabel= "days",
ylabel = "p5 sensitivity", xlims = (tspan[1],tspan[2]), dpi = 120, linewidth = 3)
SAVE_FIGURES && savefig("CARRGO6")
display(CARRGO1)
display(CARRGO2)
display(CARRGO3)
display(CARRGO4)
display(CARRGO5)
display(CARRGO6)

# Figure 3

In [ ]:
p1 = (0.48 - 0.0012) / 2 + 0.48
p2 = (0.588 - 0.417) / 2 + 0.417
(S0,I0) = (3.4e8, 100.0)
p = [p1, p2, S0+I0]; # parameters
x0 = [S0, I0, 0.0]; # initial values
(d, tspan) = (1.0e-12, (0.0, 365.0)) # 365 days
solution = analytic_method(Order1(), SIR, x0, p, tspan, d)
Covid = plot(solution[1][:, 1:3], label = ["x1" "d1x1" "d2x1"],
xlabel= "days", xlims = (tspan[1],tspan[2]), dpi = 120, linewidth = 3)
SAVE_FIGURES && savefig("Covid")
display(Covid)

# Figure 4

In [ ]:
#=please note that due to the random nature of the parameter perturbations, running this cell may not result in exactly the
same figure as in the manuscript=#

t = 100
function differential(f::Function, p, d)
    fvalue = real(f(p)) # function value
    df = zeros(length(fvalue), 0) # empty differential
    for i = 1:length(p)
        p_i = p[i]
        p[i] = p_i + d * im # perturb parameter
        fi = f(p) # compute perturbed function value
        p[i] = p_i # reset parameter to p_i + 0*i
        df  = [df imag(fi) ./ d] # concatenate ith partial
    end
    return (fvalue, df) # return the differential
end
function hessian_2(f::Function, p, d)
    n = t+1
    d2f = zeros(length(p), length(p), n) # hessian
    dp = d * (1.0 + 1.0 * im) / sqrt(2) # perturbation
    for i = 1:length(p) # compute diagonal entries of d2f
        p_i = p[i]
        p[i] = p_i + dp
        fplus = f(p)[1]
        p[i] = p_i - dp
        fminus = f(p)[1]
        p[i] = p_i
        d2f[i, i,:]  = imag(fplus + fminus) / d^2
    end
    for j = 2:length(p) # compute off diagonal entries of d2f
        for k = 1:(j - 1)
            (p_j, p_k) = (p[j], p[k])
            (p[j], p[k]) = (p_j + dp, p_k + dp)
            fplus = f(p)[1]
            (p[j], p[k]) = (p_j - dp, p_k - dp)
            fminus = f(p)[1]
            (p[j], p[k]) = (p_j, p_k)
            d2f[j, k,:]  = imag(fplus + fminus) / d^2
            d2f[j, k,:] = (d2f[j, k,:] - d2f[j, j,:] - d2f[k, k,:]) / 2
            d2f[k, j,:] = d2f[j, k,:]
        end
    end
    return d2f
end
function solve_SIR_S(p)
    epsilon1 = 1e-12
    epsilon2 = 1e-6
    N = 1000
    n = 10
    x0 = complex([N - n, n, 0.0])
    params = complex([p[1], p[2], N])
    tspan = (0.0,t)
    ODE = SIR
    problem = ODEProblem(ODE, x0, tspan, params)
    sol = solve(problem, saveat = 1.0)
    solution = [sol[1, :]]
    return solution
end
function solve_SIR_I(p)
    epsilon1 = 1e-12
    epsilon2 = 1e-6
    N = 1000
    n = 10
    x0 = complex([N - n, n, 0.0])
    params = complex([p[1], p[2], N])
    tspan = (0.0,t)
    ODE = SIR
    problem = ODEProblem(ODE, x0, tspan, params)
    sol = solve(problem, saveat = 1.0)
    solution = [sol[2, :]]
    return solution
end
function solve_SIR_R(p)
    epsilon1 = 1e-12
    epsilon2 = 1e-6
    N = 1000
    n = 10
    x0 = complex([N - n, n, 0.0])
    params = complex([p[1], p[2], N])
    tspan = (0.0,t)
    ODE = SIR
    problem = ODEProblem(ODE, x0, tspan, params)
    sol = solve(problem, saveat = 1.0)
    solution = [sol[3, :]]
    return solution
end
#=
The pseudorandom stream for MersenneTwister is liable to change across Julia versions.
Use a local RNG with seed = 1903. Should make results reproducible for a given Julia version.
=#
RNG = MersenneTwister(1903)
p = [0.105, 0.12]
change = (rand(RNG, length(p)) .- .5) .* (.25/.5)
h = change .*p
p_new = p+h
f = [solve_SIR_S, solve_SIR_I, solve_SIR_R]
ylabs = ["Susceptibles", "Infecteds", "Recovereds"]
for j = 1:3
    (fx, dfx) = differential(f[j], complex(p), 1e-12)
    first = fx[1] + [dfx[1] dfx[2]] * h
    d2f = hessian_2(f[j], complex(p), 1e-6)
    second = zeros(t+1)
    for i = 1:(t+1)
        second[i] = first[i] + .5 * transpose(h) * d2f[:,:,i]*h
    end
    actual = real(f[j](p_new))[1]
    original = real(f[j](p))[1]
    println("Euclidean distance between first order prediction and actual trajectory = ", sqrt(sum((actual - first).^2)))
    println("Euclidean distance between second order prediction and actual trajectory = ", sqrt(sum((actual - second).^2)))
    trajectory = plot(coalesce(0:t), original, label = "original", xlab = "days", ylab = ylabs[j], dpi = 120, linewidth = 5)
    plot!(trajectory, actual, label="actual", linewidth = 5)
    plot!(trajectory, first, label = "first order prediction", linewidth = 3, ls=:dash)
    plot!(trajectory, second, label = "second order prediction", linewidth = 3, ls=:dash)
    SAVE_FIGURES && savefig(ylabs[j])
    display(trajectory)
end

# Figure 5

In [ ]:
N=100
n=Int(floor(N/100))
iters=10
delta=zeros(iters)
eta=zeros(iters)
m=zeros(iters,iters)
dmdd=zeros(iters,iters)
dmde=zeros(iters,iters)
t=zeros(iters,iters)
dtdd=zeros(iters,iters)
dtde=zeros(iters,iters)
for i in 1:iters
    delta[i]=4e-2 + (i-1)*(6e-2 - 4e-2)/iters #evaluate on increments in the range 0.04 to 0.06
    for j in 1:iters
        eta[j]=1e-3 + (j-1)*(5e-1 - 1e-3)/iters #evaluate on increments in the range 0.001 to 0.5
        m[i,j]=ASPkg.meannumber(N,delta[i],eta[j])[n+1]
        dmde[i,j]=ASPkg.dm_deta(N,delta[i],eta[j])[n+1]
        dmdd[i,j]=ASPkg.dm_ddelta(N,delta[i],eta[j])[n+1]
        t[i,j]=ASPkg.meantime(N,delta[i],eta[j])[n+1]
        dtde[i,j]=ASPkg.dt_deta(N,delta[i],eta[j])[n+1]
        dtdd[i,j]=ASPkg.dt_ddelta(N,delta[i],eta[j])[n+1]
    end
end
mplot=Plots.heatmap(delta, eta, m,xaxis="delta",yaxis="eta",title="Number Infected")
SAVE_FIGURES && savefig("mplot")
dmdeplot=Plots.heatmap(delta, eta, dmde,xaxis="delta",yaxis="eta",title="dM/deta")
SAVE_FIGURES && savefig("dmdeplot")
dmddplot=Plots.heatmap(delta, eta, dmdd,xaxis="delta",yaxis="eta",title="dM/ddelta")
SAVE_FIGURES && savefig("dmddplot")
tplot=Plots.heatmap(delta, eta, t,xaxis="delta",yaxis="eta",title="Days to Extinction")
SAVE_FIGURES && savefig("tplot")
dtdeplot=Plots.heatmap(delta, eta, dtde,xaxis="delta",yaxis="eta",title="dT/deta")
SAVE_FIGURES && savefig("dtdeplot")
dtddplot=Plots.heatmap(delta, eta, dtdd,xaxis="delta",yaxis="eta",title="dT/ddelta")
SAVE_FIGURES && savefig("dtddplot")
display(mplot)
display(dmdeplot)
display(dmddplot)
display(tplot)
display(dtdeplot)
display(dtddplot)

# Table 1

In [ ]:
#=Please note that due to the random nature of stochastic simulation, running this cell may not result in exactly the same
simulated values as appears in the manuscript.=#

#model parameters we could change
N = 34000
t = 10000
numSim = 100
n = 1

#model parameters to keep simulation consistent
eta = (0.48 - 0.0012) / 2 + 0.48
delta = (0.588 - 0.417) / 2 + 0.417
extinctionTimes = zeros(numSim)
numberInfected = zeros(numSim)
i = 1
numSkipped = 0

network = Network("SIR")
network <= Species("S", N - n)
network <= Species("I", n)
network <= Species("R", 0)
network <= BioSimulator.Reaction("infection", eta/N, "S + I --> I + I")
network <= BioSimulator.Reaction("immunity", delta, "I --> R")
state, model = parse_model(network::Network)

Random.seed!(1903) # BioSimulator uses GLOBAL_RNG; this makes result here semi-reproducible
while i <= numSim
    trajectory = simulate(state, model, BioSimulator.SortingDirect(), tfinal = t, save_points = 0:t)
    if trajectory[t][2] > 0
        numSkipped += 1
    else
        numberInfected[i] = trajectory[t + 1][3]
        j = 1
        while trajectory[j][2] > 0
            j = j+1
        end
        extinctionTimes[i] = j
        i += 1
    end
end

println(numSkipped, " trajectories did not go extinct by tEnd")

#statistics for mean number
std_m = Statistics.std(numberInfected)
mean_m = mean(numberInfected)
ste_m = std_m / sqrt(numSim)
calc_m = ASPkg.meannumber(N, delta, eta)[n+1]
println("Mean number ever infected: ", mean_m)
println("Standard error for number: ", ste_m)
println("Range of one standard error for number: [", mean_m - ste_m, ", ", mean_m + ste_m, "]")
println("Calculated mean number: ", calc_m)

#statistics for mean time
std_t = Statistics.std(extinctionTimes)
mean_t = mean(extinctionTimes)
ste_t = std_t / sqrt(numSim)
calc_t = ASPkg.meantime(N, delta, eta)[n+1]
println("Mean time to extinction: ", mean_t)
println("Standard error for time: ", ste_t)
println("Range of one standard error for time: [", mean_t - ste_t, ", ", mean_t + ste_t, "]")
println("Calculated mean time: ", calc_t)

# Table 2

In [57]:
#=Note - code has been updated to use some of the suggested functions / tolerances, which causes both methods to perform
better than previously reported.  Difference in performance however is decreased.=#

#=Note - in order for multithreading to work on a Jupyter notebook, the notebook should be run in the following way:
ENV["JULIA_NUM_THREADS"] = 4
using IJulia
notebook()=#
N = 3.4e8
n = 100
change = 0.1
x0 = [N - n, n, 0.0]
params = [0.105, 0.12, N]
system = SIR
alg = AutoVern9(Rodas5(autodiff=false))
tol = 1e-12
kwargs = (alg=alg, abstol=tol, reltol=tol)
sensealg=nothing
epsilon1 = 1e-12
epsilon2 = params * 1e-6
clamp!(epsilon2, 1e-8, 1e-1) # restrict ϵ₂ to interval [1e-8, 1e-1]

accuracy_10 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 10; kwargs...)
time_10 = benchmarkSIR(10, N, n, change, sensealg, epsilon1; kwargs...)
accuracy_100 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 100; kwargs...)
time_100 = benchmarkSIR(100, N, n, change, sensealg, epsilon1; kwargs...)
accuracy_1000 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 1000; kwargs...)
time_1000 = benchmarkSIR(1000, N, n, change, sensealg, epsilon1; kwargs...)

SIR_data = DataFrame(functions = ["FD.jl 1st", "FD 1st chunk 1", "DES.jl 1st", "Analytic 1st", "Analytic 1st Multi", "FD 1st Multi", "FD 1st Chunk 1 Multi", "FD.jl 2nd (Hessian)", "FD.jl 2nd (Jacobian)", "FD 2nd Chunk 1", "DES.jl 2nd", "Nonadjoint 2nd", "Analytic 2nd",
                        "Analytic 2nd Multi", "FD 2nd Multi", "FD 2nd Chunk 1 Multi"], time_10 = time_10, time_100 = time_100, time_1000 = time_1000, 
                        accuracy_10 = accuracy_10, accuracy_100 = accuracy_100, accuracy_1000 = accuracy_1000)
show(SIR_data, allcols=true)

16×7 DataFrame
 Row │ functions             time_10   time_100        time_1000      accuracy_10  accuracy_100  accuracy_1000 
     │ String                Float64   Float64         Float64        Float64      Float64       Float64       
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ FD.jl 1st               139.5      433.25        3197.65          1.19471       274.748         9864.72
   2 │ FD 1st chunk 1          340.6     1096.2         8734.4           1.19471       274.748         9864.72
   3 │ DES.jl 1st               80.2      229.8         1767.6           1.19471       274.749         9864.72
   4 │ Analytic 1st            243.75     993.75        8819.1           1.19471       274.749         9864.72
   5 │ Analytic 1st Multi      248.5     1032.6        12156.6           1.19471       274.749         9864.72
   6 │ FD 1st Multi            146.7      439.0         4792.8           1.19471       274.748

In [5]:
#SIR model with sensealg = ForwardDiffOverAdjoint(QuadratureAdjoint(autojacvec=EnzymeVJP()))
#=Note - I am having trouble getting second_order_sensitivities to work with this sensealg; it just returns zeros.  Not sure
quite how to implement this.=#

N = 3.4e8
n = 100
change = 0.1
x0 = [N - n, n, 0.0]
params = [0.105, 0.12, N]
system = SIR
alg = AutoVern9(Rodas5(autodiff=false))
tol = 1e-12
kwargs = (alg=alg, abstol=tol, reltol=tol)
sensealg = ForwardDiffOverAdjoint(QuadratureAdjoint(autojacvec=EnzymeVJP()))
epsilon1 = 1e-12
epsilon2 = params * 1e-6
clamp!(epsilon2, 1e-8, 1e-1) # restrict ϵ₂ to interval [1e-8, 1e-1]
time_1000 = benchmarkSIR(1000, N, n, change, sensealg, epsilon1; kwargs...)
time_100 = benchmarkSIR(100, N, n, change, sensealg, epsilon1; kwargs...)
time_10 = benchmarkSIR(10, N, n, change, sensealg, epsilon1; kwargs...)
accuracy_1000 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 1000; kwargs...)
accuracy_100 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 100; kwargs...)
accuracy_10 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 10; kwargs...)

SIR_data_2 = DataFrame(functions = ["FD.jl 1st", "FD 1st chunk 1", "DES.jl 1st", "Analytic 1st", "Analytic 1st Multi", "FD 1st Multi", "FD 1st Chunk 1 Multi", "FD.jl 2nd (Hessian)", "FD.jl 2nd (Jacobian)", "FD 2nd Chunk 1", "DES.jl 2nd", "Nonadjoint 2nd", "Analytic 2nd",
                        "Analytic 2nd Multi", "FD 2nd Multi", "FD 2nd Chunk 1 Multi"], time_10 = time_10, time_100 = time_100, time_1000 = time_1000, 
                        accuracy_10 = accuracy_10, accuracy_100 = accuracy_100, accuracy_1000 = accuracy_1000)
show(SIR_data_2, allcols=true)

16×7 DataFrame
 Row │ functions             time_10   time_100       time_1000      accuracy_10  accuracy_100  accuracy_1000 
     │ String                Float64   Float64        Float64        Float64      Float64       Float64       
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ FD.jl 1st               191.5     600.0         3311.7           1.19471       274.748         9864.72
   2 │ FD 1st chunk 1          420.7    1470.2        10364.2           1.19471       274.748         9864.72
   3 │ DES.jl 1st               89.7     307.0         1650.1           1.19471       274.749         9864.72
   4 │ Analytic 1st            312.0    1353.25        8453.4           1.19471       274.749         9864.72
   5 │ Analytic 1st Multi      234.2    1149.3         6725.3           1.19471       274.749         9864.72
   6 │ FD 1st Multi            172.0     601.0         3272.4           1.19471       274.748         

In [6]:
#SIR model with eps(T) and sqrt(eps(T)) used
N = 3.4e8
n = 100
x0 = [N - n, n, 0.0]
params = [0.105, 0.12, N]
system = SIR
change = 0.1
alg = AutoVern9(Rodas5(autodiff=false))
tol = 1e-12
kwargs = (alg=alg, abstol=tol, reltol=tol)
sensealg=nothing
T = typeof(params[1])
epsilon1 = eps(T)
epsilon2 = zeros(T, length(params))
fill!(epsilon2, sqrt(eps(T)))
time_1000 = benchmarkSIR(1000, N, n, change, sensealg, epsilon1; kwargs...)
time_100 = benchmarkSIR(100, N, n, change, sensealg, epsilon1; kwargs...)
time_10 = benchmarkSIR(10, N, n, change, sensealg, epsilon1; kwargs...)
accuracy_1000 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 1000; kwargs...)
accuracy_100 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 100; kwargs...)
accuracy_10 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 10; kwargs...)

SIR_data_3 = DataFrame(functions = ["FD.jl 1st", "FD 1st chunk 1", "DES.jl 1st", "Analytic 1st", "Analytic 1st Multi", "FD 1st Multi", "FD 1st Chunk 1 Multi", "FD.jl 2nd (Hessian)", "FD.jl 2nd (Jacobian)", "FD 2nd Chunk 1", "DES.jl 2nd", "Nonadjoint 2nd", "Analytic 2nd",
                        "Analytic 2nd Multi", "FD 2nd Multi", "FD 2nd Chunk 1 Multi"], time_10 = time_10, time_100 = time_100, time_1000 = time_1000, 
                        accuracy_10 = accuracy_10, accuracy_100 = accuracy_100, accuracy_1000 = accuracy_1000)
show(SIR_data_3, allcols=true)

16×7 DataFrame
 Row │ functions             time_10  time_100       time_1000      accuracy_10  accuracy_100  accuracy_1000 
     │ String                Float64  Float64        Float64        Float64      Float64       Float64       
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ FD.jl 1st               188.8    629.05        4469.1           1.19471       274.748         9864.72
   2 │ FD 1st chunk 1          449.3   1561.55       12545.2           1.19471       274.748         9864.72
   3 │ DES.jl 1st               88.6    289.8         2224.75          1.19471       274.749         9864.72
   4 │ Analytic 1st            318.5   1396.25       12762.8           1.19471       274.749         9864.72
   5 │ Analytic 1st Multi      242.8   1214.7         9836.15          1.19471       274.749         9864.72
   6 │ FD 1st Multi            182.9    598.9         4466.3           1.19471       274.748         9864.72
 

In [58]:
#CARRGO model
x0 = [1.25e4, 6.25e2] 
params = [6.0e-9, 3.0e-11, 1.0e-6, 6.0e-2, 1.0e9]
system = CARRGO
alg = AutoVern9(Rodas5(autodiff=false))
sensealg=nothing
tol = 1e-12
kwargs = (alg=alg, abstol=tol, reltol=tol, alg_hints=[:stiff])
epsilon1 = 1e-12
epsilon2 = params * 1e-6
clamp!(epsilon2, 1e-8, 1e-1) # restrict ϵ₂ to interval [1e-8, 1e-1]
change = 0.1
time_1000 = benchmarkCARRGO(1000, change, sensealg, epsilon1; kwargs...)
time_100 = benchmarkCARRGO(100, change, sensealg, epsilon1; kwargs...)
time_10 = benchmarkCARRGO(10, change, sensealg, epsilon1; kwargs...)
accuracy_1000 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 1000; kwargs...)
accuracy_100 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 100; kwargs...)
accuracy_10 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 10; kwargs...)

CARRGO_data = DataFrame(functions = ["FD.jl 1st", "FD 1st chunk 1", "DES.jl 1st", "Analytic 1st", "Analytic 1st Multi", "FD 1st Multi", "FD 1st Chunk 1 Multi", "FD.jl 2nd (Hessian)", "FD.jl 2nd (Jacobian)", "FD 2nd Chunk 1", "DES.jl 2nd", "Nonadjoint 2nd", "Analytic 2nd",
                        "Analytic 2nd Multi", "FD 2nd Multi", "FD 2nd Chunk 1 Multi"], time_10 = time_10, time_100 = time_100, time_1000 = time_1000, 
                        accuracy_10 = accuracy_10, accuracy_100 = accuracy_100, accuracy_1000 = accuracy_1000)
show(CARRGO_data, allcols=true)

16×7 DataFrame
 Row │ functions             time_10   time_100       time_1000      accuracy_10  accuracy_100   accuracy_1000 
     │ String                Float64   Float64        Float64        Float64      Float64        Float64       
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ FD.jl 1st               371.6    1485.1        10999.9           6.19566       2.80577e5      1.46599e8
   2 │ FD 1st chunk 1          583.3    2335.6        19224.4           6.19566       2.80577e5      1.46598e8
   3 │ DES.jl 1st              120.9     369.1         2296.75          6.19476       2.80116e5      1.46492e8
   4 │ Analytic 1st            441.0    2142.95       17607.2           6.19476       2.80116e5      1.46447e8
   5 │ Analytic 1st Multi      443.45   2172.2        25804.5           6.19476       2.80116e5      1.46447e8
   6 │ FD 1st Multi            460.3    1485.1        11027.9           6.19566       2.80577e

In [ ]:
x0 = [0.1, 0.05, 0.01, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
params = [0.05, 0.025, 0.4, 0.1, 0.175, 1.5, 0.2, 0.1, 1.0, 0.1, 0.005, 0.1, 2.0, 0.4, 0.15]
system = MCC
alg = AutoVern9(Rodas5(autodiff=false))
sensealg=nothing
tol = 1e-12
kwargs = (alg=alg, abstol=tol, reltol=tol)
epsilon1 = 1e-12
epsilon2 = params * 1e-6
clamp!(epsilon2, 1e-8, 1e-1) # restrict ϵ₂ to interval [1e-8, 1e-1]
change = 0.1

time_10 = benchmarkMCC(10, change, sensealg, epsilon1, true, true; kwargs...)
time_100 = benchmarkMCC(100, change, sensealg, epsilon1, true, true; kwargs...)
time_1000 = benchmarkMCC(1000, change, sensealg, epsilon1, true, true; kwargs...)
accuracy_10 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 10, true, true; kwargs...)
accuracy_100 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 100, true, true; kwargs...)
accuracy_1000 = ODEAccuracy(system, x0, params, sensealg, epsilon1, epsilon2, 1000, true, true; kwargs...)

MCC_data = DataFrame(functions = ["FD.jl 1st", "FD 1st chunk 1", "DES.jl 1st", "Analytic 1st", "Analytic 1st Multi", "FD 1st Multi", "FD 1st Chunk 1 Multi", "FD.jl 2nd (Hessian)", "FD.jl 2nd (Jacobian)", "FD 2nd Chunk 1", "DES.jl 2nd", "Nonadjoint 2nd", "Analytic 2nd",
                        "Analytic 2nd Multi", "FD 2nd Multi", "FD 2nd Chunk 1 Multi"], time_10 = time_10, time_100 = time_100, time_1000 = time_1000, 
                        accuracy_10 = accuracy_10, accuracy_100 = accuracy_100, accuracy_1000 = accuracy_1000)
show(MCC_data, allcols=true)

┌ Warning: #= C:\Users\meste\.julia\packages\RecursiveFactorization\7pQ6n\src\lu.jl:229 =#:
│ `LoopVectorization.check_args` on your inputs failed; running fallback `@inbounds @fastmath` loop instead.
│ Use `warn_check_args=false`, e.g. `@turbo warn_check_args=false ...`, to disable this warning.
└ @ RecursiveFactorization C:\Users\meste\.julia\packages\LoopVectorization\kRxy5\src\condense_loopset.jl:821
┌ Warning: #= C:\Users\meste\.julia\packages\RecursiveFactorization\7pQ6n\src\lu.jl:237 =#:
│ `LoopVectorization.check_args` on your inputs failed; running fallback `@inbounds @fastmath` loop instead.
│ Use `warn_check_args=false`, e.g. `@turbo warn_check_args=false ...`, to disable this warning.
└ @ RecursiveFactorization C:\Users\meste\.julia\packages\LoopVectorization\kRxy5\src\condense_loopset.jl:821


# Table 3

In [ ]:
vals_time = ASPkg.benchmarkSIR_bySizeEta()
vals_accuracy = ASPkg.SIRAccuracyBySizeEta(.1)
total_data = DataFrame(method = repeat(["dmdη","dmdη_complex", "dtdη", 
            "dtdη_complex"], 3), N = vcat(repeat([10], 4), repeat([100], 4), repeat([1000], 4)), 
            times = vcat(vals_time[1, 3], vals_time[4, 3], vals_time[7, 3], vals_time[10, 3], vals_time[2, 3], 
            vals_time[5, 3], vals_time[8, 3], vals_time[11, 3], vals_time[3, 3], vals_time[6, 3], vals_time[9, 3], 
            vals_time[12, 3]), distance = vec(transpose(vals_accuracy[:,2:5])))

# Table 4

In [ ]:
vals_time = ASPkg.benchmarkBranching_bySizeBeta()
vals_accuracy = ASPkg.branchingAccuracyBySizeBeta(.1)
total_data = DataFrame(method = repeat(["dAdβ_inverse","dAdβ_inverse_analytic", "dedβ_iterative", 
            "dedβ_iterative_analytic"], 3), N = vcat(repeat([10], 4), repeat([100], 4), repeat([1000], 4)), 
            times = vals_time[:,3], distance = vcat(vals_accuracy[1, 2:3], vals_accuracy[1, 5:6], 
            vals_accuracy[2, 2:3], vals_accuracy[2, 5:6], vals_accuracy[3, 2:3], vals_accuracy[3, 5:6]))

# Code from Manuscript

In [ ]:
#page 5 (used in Figure 4)
function differential(f::Function, p, d)
    fvalue = real(f(p)) # function value
    df = zeros(length(fvalue), 0) # empty differential
    for i = 1:length(p)
        p[i] = p[i] + d * im # perturb parameter
        fi = f(p) # compute perturbed function value
        p[i] = complex(real(p[i]), 0.0) # reset parameter to p_i + 0*i
        df  = [df imag(fi) ./ d] # concatenate ith partial
    end
    return (fvalue, df) # return the differential
end

In [ ]:
#page 7 (adapted for use in Figure 4)
function hessian(f::Function, p, d)
    d2f = zeros(length(p), length(p)) # hessian
    dp = d * (1.0 + 1.0 * im) / sqrt(2) # perturbation
    for i = 1:length(p) # compute diagonal entries of d2f
        p[i] = p[i] + dp
        fplus = f(p)
        p[i] = p[i] - 2 * dp
        fminus = f(p)
        p[i] = p[i] + dp
        d2f[i, i]  = imag(fplus + fminus) / d^2
    end
    for j = 2:length(p) # compute off diagonal entries of d2f
        for k = 1:(j - 1)
            (p[j], p[k]) = (p[j] + dp, p[k] + dp)
            fplus = f(p)
            (p[j], p[k]) = (p[j] - 2 * dp, p[k] - 2 * dp)
            fminus = f(p)
            (p[j], p[k]) = (p[j] + dp, p[k] + dp)
            d2f[j, k]  = imag(fplus + fminus) / d^2
            d2f[j, k] = (d2f[j, k] - d2f[j, j] - d2f[k, k]) / 2
            d2f[k, j] = d2f[j, k]
        end
    end
    return d2f
end

In [ ]:
#page 12
function sensitivity(x0, p, d, tspan)
  problem = ODEProblem{true}(ODE, x0, tspan, p)
  sol = solve(problem, saveat = 1.0) # solve ODE
  (lp, ls, lx) = (length(p), length(sol), length(x0))  
  solution = Dict{Int, Any}(i => zeros(ls, lp + 1) for i in 1:lx)
  for j = 1:lx # record solution for each species
    @views solution[j][:, 1] = sol[j, :]
  end
  for j = 1:lp
    p[j] = p[j] + d * im # perturb parameter
    problem = ODEProblem{true}(ODE, x0, tspan, p)
    sol = solve(problem, saveat = 1.0) # resolve ODE
    p[j] = complex(real(p[j]), 0.0) # reset parameter
    @views sol .= imag(sol) / d # compute partial
    for k = 1:lx # record partial for each species
      @views solution[k][:,j + 1] = sol[k, :]
    end
  end
  return solution
end

function ODE(dx, x, p, t) # CARRGO model
  dx[1] = p[4] * x[1] * (1 - x[1] / p[5]) - p[1] * x[1] * x[2]
  dx[2] = p[2]* x[1] * x[2] - p[3] * x[2]
end

p = complex([6.0e-9, 3.0e-11, 1.0e-6, 6.0e-2, 1.0e9]); # parameters
x0 = complex([1.25e4, 6.25e2]); # initial values
(d, tspan) = (1.0e-13, (0.0,1000.0)); # step size and time interval in days
solution = sensitivity(x0, p, d, tspan); # find solution and partials
CARRGO1 = plot(solution[1][:, 1], label = "x1", xlabel= "days", 
ylabel = "cancer cells x1", xlims = (tspan[1],tspan[2]))
CARRGO2 = plot(solution[1][:, 2], label = "d1x1", xlabel= "days", 
ylabel = "p1 sensitivity", xlims = (tspan[1],tspan[2]))

In [ ]:
#page 13
function ODE(dx, x, p, t) # Covid model
    N = 3.4e8 # US population size
    dx[1] = - p[1] * x[2] * x[1] / N
    dx[2] = p[1] * x[2] * x[1] / N - p[2] * x[2]
    dx[3] = p[2] * x[2]
end
p = complex([0.2, (0.0417 + 0.0588) / 2]); # parameters
x0 = complex([3.4e8, 100.0, 0.0]); # initial values
(d, tspan) = (1.0e-10, (0.0, 365.0)) # 365 days
solution = sensitivity(x0, p, d, tspan)
Covid = plot(solution[1][:, :], label = ["x1" "d1x1" "d2x1"],xlabel = "days", xlims = (tspan[1],tspan[2]))

In [ ]:
#page 16
function SIRMeans(p)
    (delta, eta) = (p[1], p[2])
    M = Matrix(zeros(eltype(p), N + 1, N + 1)) # mean matrix
    T = similar(M) # time to extinction matrix
    for n = 1:N # recurrence relations loop
        for j = 0:(n - 1)
            i = n - ja = i * delta # immunity rate
            if i == n # initial conditions
                M[i + 1, n + 1] = i
                T[i + 1, n + 1] = T[i, i] + 1 / a
            else
                b = i * (n - i) * eta / N  # infection rate
                c = 1 / (a + b)
                M[i + 1, n + 1] = a * c * (M[i, n] + 1) + b * c * M[i + 2, n + 1]
                T[i + 1, n + 1] = c * (1 + a * T[i, n] + b * T[i + 2, n + 1])
            end
        end
    end
    return [M[:, N + 1]; T[:, N + 1]]
end

In [ ]:
#page 22
function extinction(p)
    types = Int(sqrt(1 + length(p)) - 1) #length(p) = 2* types + types^2 
    (x, y) = (zeros(Complex, types), zeros(Complex, types))
    for i = 1:500 # functional iteration
        y = P(x, p)
        if norm(x - y) < 1.0e-16 
            break 
        end
        x = copy(y)
    end
    return y
end
function P(x, p) # progeny generating function
    types = Int(sqrt(1 + length(p)) - 1) #length(p) = 2* types + types^2 
    delta = p[1: types]
    beta = p[types + 1: 2 * types]
    lambda = reshape(p[2 * types + 1:end], (types, types))
    y = similar(x)
    t = delta[1] + beta[1] + lambda[1, 2]
    y[1] = (delta[1] + beta[1] * x[1]^2 + lambda[1, 2] * x[2]) / t
    t = delta[2] + beta[2] + lambda[2, 1]
    y[2] = (delta[2] + beta[2] * x[2]^2 + lambda[2, 1] * x[1]) / t
    return y
end
delta = complex([1.0, 1.75]); # death rates
beta = complex([1.5, 1.5]); # birth rates
lambda = complex([0.0 0.5; 1.0 0.0]); # migration rates
p = [delta; beta; vec(lambda)]; # package parameter vector(types, d) = (2, 1.0e-10)
@time (e, de) = differential(extinction, p, d)

In [ ]:
#page 23
function particles(p) # mean particles generated
    types = Int(sqrt(1 + length(p)) - 1) #length(p) = 2* types + types^2 
    delta = p[1: types]
    beta = p[types + 1: 2 * types]
    lambda = reshape(p[2 * types + 1:end], (types, types))
    F = complex(zeros(types, types))
    t = delta[1] + beta[1] + lambda[1, 2]
    (F[1, 1], F[1, 2]) = (2 * beta[1] / t, lambda[1, 2] / t)
    t = delta[2] + beta[2] + lambda[2, 1]
    (F[2, 1], F[2, 2]) = (lambda[2, 1] / t, 2 * beta[2] / t)
    A = vec(inv(I - F)) # return as vector
end
delta = complex([1.0, 1.75]); # death rates
beta = complex([1.5, 1.5]); # birth rates
lambda = complex([0.0 0.5; 1.0 0.0]); # migration rates
p = [delta; beta; vec(lambda)]; # package parameter 
vector(types, d) = (2, 1.0e-10)
@time (A, dA) = differential(particles, p, d)